In [133]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import json
import re
from sklearn.preprocessing import StandardScaler
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
standard_X = StandardScaler()

In [112]:
dfp = pd.read_csv('data-set/personal_valid.csv')
dfo = pd.read_csv('data-set/other_valid.csv')
dfp = dfp.rename(columns={'Unnamed: 0': 'ID'})
dfo = dfo.rename(columns={'Unnamed: 0': 'ID'})

Táto časť je skopírovaná z predchadzajúceho notebooku 'Prieskumná_analýza.ipynb'. Spojí dataset other a personal do jedného.

In [113]:
# merges duplicates
def merge(name):
    df = dfo_duplicates.loc[dfo_duplicates['name'] == name]
    return df.groupby(['name'], as_index=False).first()

dfo_duplicates = dfo[dfo.duplicated(['name'], keep=False)]
dfo_unique = dfo.drop_duplicates(subset=["name"], keep=False)
merged = []

for name in dfo_duplicates['name'].unique():
    merged.append(merge(name))
    
dfo_unique = dfo_unique.append(merged)

df = pd.merge(dfp, dfo_unique, on='name')
df = df.drop(columns=['address_y', 'ID_y'])
df = df.rename(columns={"ID_x": "ID", "address_x": "address"})

Pridame nove stlpce na rozbalenie hodnot v medical_info.

In [114]:
for i, row in df.iterrows():
    if not pd.isnull(df.at[i, 'medical_info']):
        df.at[i, 'medical_info'] = json.loads(df["medical_info"][i].replace("\'", "\""))

In [115]:
df_med_info = df["medical_info"].apply(pd.Series)
df_med_info = df_med_info.drop(0, 1)

df = pd.concat([df, df_med_info], axis = 1).drop("medical_info", axis = 1)
df

,ID,name,address,age,sex,date_of_birth,race,marital-status,occupation,pregnant,...,std_glucose,income,native-country,hours-per-week,capital-loss,workclass,kurtosis_oxygen,mean_oxygen,skewness_oxygen,std_oxygen
0,0,Philip Miller,"7910 Rosales Plain Apt. 454\r\nPort Carl, GA 6...",48,Male,1971-07-10 00 00 00,White,Married-civ-spouse,Other-service,f,...,45.326894,<=50K,United-States,40.0,0.0,Private,4.642781783999999,7.227424749,21.80403398,31.43028784
1,1,Mitch Wilson,055 Morgan Plains Suite 225\r\nEast Darrylmout...,54,Female,1965-08-09,White,Divorced,Machine-op-inspct,f,...,31.877558,<=50K,United-States,40.0,0.0,Private,1.731935856,41.42725753,2.537561569,61.23547314
2,2,James Olsen,"PSC 7359, Box 2088\r\nAPO AE 62717",48,Female,1971-01-20 00:00:00,Black,Widowed,Machine-op-inspct,f,...,43.385465,<=50K,United-States,48.0,0.0,Private,7.134172179,3.418896321,58.95355939,19.26364202
3,3,Maurice Riley,USS Allen\r\nFPO AP 87677,59,Female,1959-11-02,White,Never-married,Prof-specialty,f,...,32.258590,>50K,United-States,40.0,1564.0,Private,0.7429214190000001,68.93478261,-0.08900948300000001,65.17461055
4,4,Larry Stanley,"491 Fields Key Suite 544\r\nGracestad, MT 13456",36,Male,1983-08-13,White,Married-civ-spouse,Handlers-cleaners,f,...,50.723635,>50K,United-States,40.0,0.0,Private,9.078859855,2.269230769,89.74845566,18.35189536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1306,1306,Michael Everett,"35892 Robert Forest Apt. 556\r\nRubioside, AL ...",46,Female,1972-11-13,White,Divorced,Prof-specialty,f,...,35.656589,<=50K,United-States,35.0,0.0,Local-gov,7.701320252,3.136287625,73.0199181,17.46062903
1307,1307,Julian Patrick,"53981 Taylor Brook\r\nBarkerville, TN 17001",53,Male,1965-12-04,White,Never-married,Handlers-cleaners,f,...,31.317723,<=50K,Portugal,50.0,0.0,Private,2.669580267,22.63461538,6.700530685,50.46831954
1308,1308,Lee Walker,"847 Jamie Cove Apt. 464\r\nCummingsview, RI 90999",33,Male,1986-09-20,White,Married-civ-spouse,Sales,f,...,51.607130,>50K,United-States,40.0,0.0,Private,9.371371993,2.594481605,104.1369339,16.90377853
1309,1309,Thomas Garrison,"159 Yvonne Way Suite 696\r\nWest Ashleyton, LA...",55,Male,64-05-10,White,Never-married,Craft-repair,f,...,45.399098,<=50K,United-States,30.0,0.0,Private,5.559184745,6.393812709,30.62363238,33.97059414


Pozrieme sa na typy udajov po pridani dalsich stlpcov

In [116]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1311 entries, 0 to 1310
Data columns (total 29 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                1311 non-null   int64  
 1   name              1311 non-null   object 
 2   address           1311 non-null   object 
 3   age               1311 non-null   int64  
 4   sex               1311 non-null   object 
 5   date_of_birth     1311 non-null   object 
 6   race              1309 non-null   object 
 7   marital-status    1226 non-null   object 
 8   occupation        1311 non-null   object 
 9   pregnant          1311 non-null   object 
 10  education-num     1180 non-null   float64
 11  relationship      1310 non-null   object 
 12  skewness_glucose  1310 non-null   float64
 13  mean_glucose      1311 non-null   float64
 14  capital-gain      1308 non-null   float64
 15  kurtosis_glucose  1311 non-null   float64
 16  education         1311 non-null   object 


Zmena kurtosis_oxygen, mean_oxygen, skewness_oxygen, std_oxygen na float

In [117]:
# kurtosis_oxygen
df['kurtosis_oxygen'] = df['kurtosis_oxygen'].astype(np.float)
df['kurtosis_oxygen']

0        4.642782
1        1.731936
2        7.134172
3        0.742921
4        9.078860
          ...    
1306     7.701320
1307     2.669580
1308     9.371372
1309     5.559185
1310    12.687270
Name: kurtosis_oxygen, Length: 1311, dtype: float64

In [118]:
# mean_oxygen
df['mean_oxygen'] = df['mean_oxygen'].astype(np.float)
df['mean_oxygen']

0        7.227425
1       41.427258
2        3.418896
3       68.934783
4        2.269231
          ...    
1306     3.136288
1307    22.634615
1308     2.594482
1309     6.393813
1310     1.384615
Name: mean_oxygen, Length: 1311, dtype: float64

In [119]:
# skewness_oxygen
df['skewness_oxygen'] = df['skewness_oxygen'].astype(np.float)
df['skewness_oxygen']

0        21.804034
1         2.537562
2        58.953559
3        -0.089009
4        89.748456
           ...    
1306     73.019918
1307      6.700531
1308    104.136934
1309     30.623632
1310    196.380946
Name: skewness_oxygen, Length: 1311, dtype: float64

In [120]:
# std_oxygen
df['std_oxygen'] = df['std_oxygen'].astype(np.float)
df['std_oxygen']

0       31.430288
1       61.235473
2       19.263642
3       65.174611
4       18.351895
          ...    
1306    17.460629
1307    50.468320
1308    16.903779
1309    33.970594
1310    12.493713
Name: std_oxygen, Length: 1311, dtype: float64

Nahradenie pohlavi ' Female', ' Male' za 'Female', 'Male'.

In [121]:
df['sex'] = df['sex'].replace(' Male', 1)
df['sex'] = df['sex'].replace(' Female', 0)

Nahradenie pregnant 't' 'T' 'F' 'f' za 'True a 'False'

In [122]:
df['pregnant'] = df['pregnant'].replace(regex='(?i)f.*', value=0)
df['pregnant'] = df['pregnant'].replace(regex='(?i)t.*', value=1)

Mame tu ludi s muzskym pohlavim ktory su oznaceni ako tehotni

In [123]:
df.loc[(df['pregnant'] == 1) & (df['sex'] == 1)]

,ID,name,address,age,sex,date_of_birth,race,marital-status,occupation,pregnant,...,std_glucose,income,native-country,hours-per-week,capital-loss,workclass,kurtosis_oxygen,mean_oxygen,skewness_oxygen,std_oxygen
878,878,Louis Emanuelson,"5279 Williams Falls Suite 033\r\nTimburgh, UT ...",41,1,78-05-19,White,Divorced,Craft-repair,1,...,45.463316,<=50K,United-States,50.0,0.0,Self-emp-not-inc,10.206538,2.280936,123.238365,16.109045
1132,1132,Steven Sao,"83139 Erica Lights Apt. 701\r\nEast Billy, IN ...",50,1,1969/07/05,White,Married-civ-spouse,Machine-op-inspct,1,...,46.975340,<=50K,United-States,40.0,0.0,Private,5.787360,6.050167,32.972108,33.865623


Upravenie adresy

In [124]:
df['address'].replace(regex='\r\n', value=' ')

0         7910 Rosales Plain Apt. 454 Port Carl, GA 69839
1       055 Morgan Plains Suite 225 East Darrylmouth, ...
2                         PSC 7359, Box 2088 APO AE 62717
3                                  USS Allen FPO AP 87677
4            491 Fields Key Suite 544 Gracestad, MT 13456
                              ...                        
1306     35892 Robert Forest Apt. 556 Rubioside, AL 36512
1307             53981 Taylor Brook Barkerville, TN 17001
1308       847 Jamie Cove Apt. 464 Cummingsview, RI 90999
1309    159 Yvonne Way Suite 696 West Ashleyton, LA 46660
1310    05550 Mahoney Crest Apt. 037 Grahammouth, MO 1...
Name: address, Length: 1311, dtype: object

**Education a education-num**  
Zistujeme, co znamena education a education num. Predpoklad je, ze education-num je numericka reprezentacia education

In [125]:
# prints unique values in education
unique_edu = pd.unique(df['education'])
unique_edu

array([' Assoc-voc', ' Some-college', ' Assoc_voc', ' HS_grad',
       ' HS-grad', ' Bachelors', ' Assoc-acdm', ' Doctorate',
       ' Some_college', ' 10th', ' 7th-8th', ' Masters', ' 11th', ' 12th',
       ' Prof-school', ' 1st-4th', ' 5th-6th', ' 9th', ' 1st_4th',
       ' 7th_8th', ' Assoc_acdm', ' Prof_school', ' 5th_6th',
       ' Preschool'], dtype=object)

Najprv upravime hodnoty education, kedze mame nejednotne reprezentacie tych istych hodnot (napr. HS_grad a HS-grad)

In [126]:
# Zjednotenie reprezentacii
df['education'] = df['education'].replace(regex='(?i)_', value='-')
unique_edu = pd.unique(df['education'])
unique_edu

array([' Assoc-voc', ' Some-college', ' HS-grad', ' Bachelors',
       ' Assoc-acdm', ' Doctorate', ' 10th', ' 7th-8th', ' Masters',
       ' 11th', ' 12th', ' Prof-school', ' 1st-4th', ' 5th-6th', ' 9th',
       ' Preschool'], dtype=object)

Po zjednoteni sa pozrieme na hodnoty education-num v education

In [127]:
for item in unique_edu:
    edu_num = df.query("education == @item")["education-num"].unique()
    print(item, edu_num)

 Assoc-voc [ 1100.    11.    nan -1100.]
 Some-college [   10.    nan  1000. -1000.]
 HS-grad [   9.   nan  900. -900.]
 Bachelors [   13. -1300.  1300.    nan]
 Assoc-acdm [   12. -1200.    nan  1200.]
 Doctorate [   16. -1600.    nan]
 10th [   6.  600.   nan -600.]
 7th-8th [   4.   nan -400.  400.]
 Masters [   14.    nan -1400.  1400.]
 11th [   7.  700. -700.   nan]
 12th [800.  nan   8.]
 Prof-school [15.]
 1st-4th [ 2. nan]
 5th-6th [  3. 300.]
 9th [5.]
 Preschool [1.]


Vidime ze education-num je ciselna reprezentacia education. Vytvorime si dictionary, ktory bude priradovat education ku education-num, s tym ze nechame celociselne reprezentacie od 1 po 16.

In [128]:
def get_edu_num(edu_num):
    for num in edu_num:
        if num is None:
            continue
        elif num < 100:
            return int(num)

In [129]:
edu_to_num = {}
for item in unique_edu:
    edu_num = df.query("education == @item")["education-num"].unique()
    edu_to_num[item] = get_edu_num(edu_num)
edu_to_num

{' Assoc-voc': 11,
 ' Some-college': 10,
 ' HS-grad': 9,
 ' Bachelors': 13,
 ' Assoc-acdm': 12,
 ' Doctorate': 16,
 ' 10th': 6,
 ' 7th-8th': 4,
 ' Masters': 14,
 ' 11th': 7,
 ' 12th': 8,
 ' Prof-school': 15,
 ' 1st-4th': 2,
 ' 5th-6th': 3,
 ' 9th': 5,
 ' Preschool': 1}

Namapujeme nazvy education na cisla z dictionary

In [130]:
df["education"].map(edu_to_num)

0       11
1       10
2       10
3       11
4        9
        ..
1306    13
1307     9
1308     9
1309     7
1310    14
Name: education, Length: 1311, dtype: int64

Dropneme nepotrebny column education-num

In [131]:
df.drop(['education-num'], axis=1)

,ID,name,address,age,sex,date_of_birth,race,marital-status,occupation,pregnant,...,std_glucose,income,native-country,hours-per-week,capital-loss,workclass,kurtosis_oxygen,mean_oxygen,skewness_oxygen,std_oxygen
0,0,Philip Miller,"7910 Rosales Plain Apt. 454\r\nPort Carl, GA 6...",48,1,1971-07-10 00 00 00,White,Married-civ-spouse,Other-service,0,...,45.326894,<=50K,United-States,40.0,0.0,Private,4.642782,7.227425,21.804034,31.430288
1,1,Mitch Wilson,055 Morgan Plains Suite 225\r\nEast Darrylmout...,54,0,1965-08-09,White,Divorced,Machine-op-inspct,0,...,31.877558,<=50K,United-States,40.0,0.0,Private,1.731936,41.427258,2.537562,61.235473
2,2,James Olsen,"PSC 7359, Box 2088\r\nAPO AE 62717",48,0,1971-01-20 00:00:00,Black,Widowed,Machine-op-inspct,0,...,43.385465,<=50K,United-States,48.0,0.0,Private,7.134172,3.418896,58.953559,19.263642
3,3,Maurice Riley,USS Allen\r\nFPO AP 87677,59,0,1959-11-02,White,Never-married,Prof-specialty,0,...,32.258590,>50K,United-States,40.0,1564.0,Private,0.742921,68.934783,-0.089009,65.174611
4,4,Larry Stanley,"491 Fields Key Suite 544\r\nGracestad, MT 13456",36,1,1983-08-13,White,Married-civ-spouse,Handlers-cleaners,0,...,50.723635,>50K,United-States,40.0,0.0,Private,9.078860,2.269231,89.748456,18.351895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1306,1306,Michael Everett,"35892 Robert Forest Apt. 556\r\nRubioside, AL ...",46,0,1972-11-13,White,Divorced,Prof-specialty,0,...,35.656589,<=50K,United-States,35.0,0.0,Local-gov,7.701320,3.136288,73.019918,17.460629
1307,1307,Julian Patrick,"53981 Taylor Brook\r\nBarkerville, TN 17001",53,1,1965-12-04,White,Never-married,Handlers-cleaners,0,...,31.317723,<=50K,Portugal,50.0,0.0,Private,2.669580,22.634615,6.700531,50.468320
1308,1308,Lee Walker,"847 Jamie Cove Apt. 464\r\nCummingsview, RI 90999",33,1,1986-09-20,White,Married-civ-spouse,Sales,0,...,51.607130,>50K,United-States,40.0,0.0,Private,9.371372,2.594482,104.136934,16.903779
1309,1309,Thomas Garrison,"159 Yvonne Way Suite 696\r\nWest Ashleyton, LA...",55,1,64-05-10,White,Never-married,Craft-repair,0,...,45.399098,<=50K,United-States,30.0,0.0,Private,5.559185,6.393813,30.623632,33.970594


In [201]:
def find_state(address):
    #i = re.search('\x5cn', address).start()
    #return address[i:].split(',')[1][1:3]
    return address[-8:][:2]

In [210]:
df['address'] = df['address'].apply(find_state)

In [211]:
df

,ID,name,address,age,sex,date_of_birth,race,marital-status,occupation,pregnant,...,std_glucose,income,native-country,hours-per-week,capital-loss,workclass,kurtosis_oxygen,mean_oxygen,skewness_oxygen,std_oxygen
0,0,Philip Miller,GA,48,1,1971-07-10 00 00 00,White,Married-civ-spouse,Other-service,0,...,45.326894,<=50K,United-States,40.0,0.0,Private,4.642782,7.227425,21.804034,31.430288
1,1,Mitch Wilson,KS,54,0,1965-08-09,White,Divorced,Machine-op-inspct,0,...,31.877558,<=50K,United-States,40.0,0.0,Private,1.731936,41.427258,2.537562,61.235473
2,2,James Olsen,AE,48,0,1971-01-20 00:00:00,Black,Widowed,Machine-op-inspct,0,...,43.385465,<=50K,United-States,48.0,0.0,Private,7.134172,3.418896,58.953559,19.263642
3,3,Maurice Riley,AP,59,0,1959-11-02,White,Never-married,Prof-specialty,0,...,32.258590,>50K,United-States,40.0,1564.0,Private,0.742921,68.934783,-0.089009,65.174611
4,4,Larry Stanley,MT,36,1,1983-08-13,White,Married-civ-spouse,Handlers-cleaners,0,...,50.723635,>50K,United-States,40.0,0.0,Private,9.078860,2.269231,89.748456,18.351895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1306,1306,Michael Everett,AL,46,0,1972-11-13,White,Divorced,Prof-specialty,0,...,35.656589,<=50K,United-States,35.0,0.0,Local-gov,7.701320,3.136288,73.019918,17.460629
1307,1307,Julian Patrick,TN,53,1,1965-12-04,White,Never-married,Handlers-cleaners,0,...,31.317723,<=50K,Portugal,50.0,0.0,Private,2.669580,22.634615,6.700531,50.468320
1308,1308,Lee Walker,RI,33,1,1986-09-20,White,Married-civ-spouse,Sales,0,...,51.607130,>50K,United-States,40.0,0.0,Private,9.371372,2.594482,104.136934,16.903779
1309,1309,Thomas Garrison,LA,55,1,64-05-10,White,Never-married,Craft-repair,0,...,45.399098,<=50K,United-States,30.0,0.0,Private,5.559185,6.393813,30.623632,33.970594
